# MAR 653 Marketing Analytics
## 


Ryan Timbrook (RTIMBROO)  
DATE:
Topic: 

## 1. Objective
_____________________________________________________________________________________________



______________________________________________________________________________________________
### Coding Environment Setup
Import packages

In [1]:
# import packages for analysis and modeling
import pandas as pd #data frame operations
import numpy as np #arrays and math functions

## -- VISUALIZATION Packages
import matplotlib.pyplot as plt #2D plotting
%matplotlib inline

plt.style.use('fivethirtyeight')
import seaborn as sns #
## -- 
import os
import re
from itertools import product
from datetime import date
from datetime import time
from datetime import datetime
import warnings
from timeit import default_timer               # performance processing time
import logging                                 # logging framework
# get loggers
warnings.filterwarnings('ignore')

## Clustering
from sklearn.cluster import KMeans, SpectralClustering



In [2]:
# custome python packages
import rtimbroo_utils as rt             # custome python helper functions
import auq_42_utils as au

All the files are downloaded


In [3]:
# set global properties
notebook_file_name = 'airbnb_seattle_customer_sentiment'
report_file_name = 'airbnb_seattle_customer_sentiment'
app_name = 'airbnb_seattle_customer_sentiment'
log_level = 10 # 10-DEBUG, 20-INFO, 30-WARNING, 40-ERROR, 50-CRITICAL

# setup working directory structure
# set global properties
dataDir = './data'
outputDir = './output'
configDir = './config'
logOutDir = './logs'
imageDir = './images'
modelDir = './models'
corpusDir = './corpus'

In [4]:
# create base output directories if they don't exist
if not os.path.exists(outputDir): os.mkdir(outputDir)
if not os.path.exists(logOutDir): os.mkdir(logOutDir)
if not os.path.exists(imageDir): os.mkdir(imageDir)
if not os.path.exists(modelDir): os.mkdir(modelDir)
if not os.path.exists(corpusDir): os.mkdir(corpusDir)

In [5]:
# get current date
now = datetime.utcnow().isoformat()
now_date = re.findall('^[0-9]{4}-[0-9]{2}-[0-9]{2}',now)
now_date

['2020-03-10']

In [6]:
#a logger for troubleshooting / data exploration
logger = rt.getFileLogger(logOutDir+'/',app_name+'_'+now_date[0],level=log_level)

## 2. OBTAIN the data   
________________________________________________________________________________________________
Import external datasets for evaluation

##### Airbnb Seattle
* Customer Reviews: reviews.csv
* Host Listings: listings.csv
* Calendar Listings: calendar.csv

In [7]:
# datasets file names to load
customer_reviews = pd.read_csv(f'{dataDir}/seattle/reviews.csv', error_bad_lines=False, encoding="ISO-8859-1")
host_listings = pd.read_csv(f'{dataDir}/seattle/listings.csv', error_bad_lines=False, encoding="ISO-8859-1")
calendar_listings = pd.read_csv(f'{dataDir}/seattle/calendar.csv', error_bad_lines=False, encoding="ISO-8859-1")

In [8]:
# customer reviews info
customer_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84849 entries, 0 to 84848
Data columns (total 6 columns):
listing_id       84849 non-null int64
id               84849 non-null int64
date             84849 non-null object
reviewer_id      84849 non-null int64
reviewer_name    84849 non-null object
comments         84831 non-null object
dtypes: int64(3), object(3)
memory usage: 3.9+ MB


In [9]:
#customer_reviews.head()
customer_reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,7202016,38917982,2015-07-19,28943674,Bianca,Cute and cozy place. Perfect location to every...
1,7202016,39087409,2015-07-20,32440555,Frank,Kelly has a great room in a very central locat...
2,7202016,39820030,2015-07-26,37722850,Ian,"Very spacious apartment, and in a great neighb..."
3,7202016,40813543,2015-08-02,33671805,George,Close to Seattle Center and all it has to offe...
4,7202016,41986501,2015-08-10,34959538,Ming,Kelly was a great host and very accommodating ...


In [ ]:
host_listings.info()

In [ ]:
host_listings.head()

In [ ]:
calendar_listings.info()

In [ ]:
calendar_listings.head()

### 2.1 SCRUB / CLEAN
Clean and perform initial transformations steps of the data

In [ ]:
'''
Utility functions to help clean the datasets
'''

def getNaNIndexes(df,att):
    n = np.where(df[att].isnull()==True)
    return list(n[0])

def getNotNaNIndexes(df,att):
    n = np.where(df[att].isnull()==False)
    return list(n[0])

def set_index_true(df,att):
    not_null_indexes = getNotNaNIndexes(df,att)
    for i in not_null_indexes:
        df[att][i] = 1

def set_nan_0(df,att):
    df[att].fillna(0,inplace=True)

def set_nan_other(df,att):
    null_indexes = getNaNIndexes(df,att)
    for i in null_indexes:
        df[att][i] = 99
        
def set_nan_median(df,att):
    null_indexes = getNaNIndexes(df,att)
    d = df.copy()
    m = d[att].dropna().median()
    for i in null_indexes:
        df[att][i] = m
    
def set_other_to_99(df,att):
    n = np.where(df[att] == 'other')
    for i in list(n[0]):
        df[att][i] = 99

## Calendar Listings


In [ ]:
def convertToFloat(col):
    
    

In [ ]:
calendar_listings['price'] = calendar_listings['price'].fillna(str('0'))
calendar_listings.price[:5]


#calendar_listings['price'].astype('int64',copy=False)[:5]

In [ ]:
p = [p for p in calendar_listings['price']]

In [ ]:
re_money = re.compile('^\$\d+')
prices = []
for e in calendar_listings['price']:
    e = str(e)
    e = e.replace('\'','')
    e = e.replace(',','')
    #print(e)
    if re.match(re_money,e):
        #print(e[1:])
        prices.append(e[1:])
    else:
        prices.append(e)
    #break
new_prices = np.array(prices).astype(np.float)

In [ ]:
calendar_listings['price'] = new_prices
calendar_listings.head()

In [ ]:
calendar_listings.info()

In [ ]:
 # output file names based on date range search
cleanPath = f'{dataDir}/clean'
if not os.path.exists(cleanPath): os.makedirs(cleanPath)
calendar_listings.to_csv(f'{cleanPath}/calendar_listings.csv', index=False)


##### Limit Host Listings data:

listings [price, weekly_price, monthly_price, security_deposit, cleaning_fee, number_of_reviews, first_review, last_review, review_scores_rating, review_scores_accuracy, review_scores_cleanliness, review_scores_checkin, review_scores_communication, review_scores_location, review_scores_value, reviews_per_month, host_since, host_response_time, host_responase_rate, host_acceptance_rate, host_is_superhost, host_listing_counts, host_total_listings_count, property_type, room_type, accommodations, bathrooms, bedrooms, beds, bed_type, amenities, square_foot, availability_30, availability_60, availability_90, avaiability_365]

In [ ]:
hosts_listings_keep = ['price', 'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee', 'number_of_reviews', 'first_review', 'last_review', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication, review_scores_location, review_scores_value, reviews_per_month, host_since, host_response_time, host_responase_rate, host_acceptance_rate, host_is_superhost, host_listing_counts, host_total_listings_count, property_type, room_type, accommodations, bathrooms, bedrooms, beds, bed_type, 'amenities', 'square_foot', 'availability_30', 'availability_60', 'availability_90', 'avaiability_365']
new_host_listings = host_listings[]
new_host_listings.head()

#### Customer Reviews data:

reviews [reviewer_name, comments, date] (keys to link the datasets will also be included such as 'listing_id', 'reviewer_id')

#### Run Wordcloud Visualizations

In [ ]:
rt.wordcloud_draw(customer_reviews['comments'], color='white',max_words=200)

   ## Save Datasets Clean

In [ ]:
 # output file names based on date range search
cleanPath = f'{dataDir}/clean'
if not os.path.exists(cleanPath): os.makedirs(cleanPath)
customer_reviews.to_csv(f'{cleanPath}/customer_reviews.csv', index=False)
host_listings.to_csv(f'{cleanPath}/host_listings.csv', index=False)

## Explore Cleaned Datasets

In [ ]:
customer_reviews = pd.read_csv(f'{dataDir}/clean/customer_reviews.csv', error_bad_lines=False, encoding="ISO-8859-1")
host_listings = pd.read_csv(f'{dataDir}/clean/host_listings.csv', error_bad_lines=False, encoding="ISO-8859-1")

In [ ]:
# transform factor attributes to categorical datatypes

# food_consumption_habbit - both - food_consumption_habbit_features
#customer_survey[food_consumption_habbit_features] = customer_survey[food_consumption_habbit_features].astype('category')
#noncustomer_survey[food_consumption_habbit_features] = noncustomer_survey[food_consumption_habbit_features].astype('category')



## Correlation Heatmaps

## 3. MODEL
_________________________________________________________________________________________________
Create models

### 3.1 Model Analysis

####3.1.1 - Segmentation Analysis



## 3.1 K-means Clustering
Python package: scikit-learn v0.21.3 [sklearn.cluster.KMeans](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans)  

Description: ...

### 3.1.3 Model - KMeans
* Run multiple k means to determin optimal k size for final model creation
    * 8 iterations were ran, where k 4 was the most optimal

In [ ]:
new_customer_survey = customer_survey.drop(columns=drop_list)

#new_customer_survey.info()
#for f in new_customer_survey.columns:
    #set_other_to_99(new_customer_survey,f)

In [ ]:
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_samples, silhouette_score

def build_kmeans(n_clusters,random_state,n_jobs,verbose=0):
    km = KMeans(
        n_clusters=n_clusters, 
        init="k-means++", # using elbow to figure out k for kmeans
        n_init=10, 
        max_iter=300, 
        tol=0.0001, 
        precompute_distances="auto", 
        verbose=verbose, 
        random_state=random_state, # determines random number generation for centroid initialization, Use an int to make the randomness deterministic, (i.e. running it multiple times will produce the same result)
        copy_x=True, 
        n_jobs=n_jobs, 
        algorithm="auto")
    
    return km

In [ ]:
# Run multiple k means to determin optimal k size for final model creation

sse = {} # store output for analysis
# set range from 2 - 20, assum max number of clusters to be 20
for k in range(2,20):
    km = build_kmeans(n_clusters=k,random_state=0,n_jobs=None)
    X_std = km.fit_transform(new_customer_survey) #
    kmeans = km.fit(X_std)
    sse[k] = kmeans.inertia_ # Inertia: sum of distances of samples to their closest cluster center

# plot elbow    
plt.figure()
plt.plot(list(sse.keys()), list(sse.values()))
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('SSE')
plt.show()

In [ ]:
logger.info(f'kmeans class labels... {kmeans.labels_}')
logger.info(f'kmeans parameters... {kmeans.get_params()}')
logger.info(f'sse out analysis, multiple k-means trials\n: {list(sse.keys()), list(sse.values())}')

In [ ]:
##### Build KMeans based on ideal cluster state found by Elbow method: ???


In [ ]:
# build KMeans based on ideal cluster state found above - 4


#### 3.1.4 Results


### 3.2 Model Exploration

##### 3.2.1 

In [ ]:
c

##### 3.2. Which Profiles should Sticks Profile?

### 3.3 Model Build

### 3.4 Model Evaluation

### 3.5 Model Results

#### Model Validation
Perform model validations

### iNterpret
Interpret the model results, make knowledge based recommendations

In [ ]:
# perform interpretation steps

# Report Generation
Execute the below cell to create an HTML report

In [ ]:
# for report auto generation
# testing markup report generation
from nbconvert import HTMLExporter
import codecs
import os
import datetime

stamp = datetime.date.today().strftime("%m-%d-%Y")
exporter = HTMLExporter(template_file='report.tpl')
output, resources = exporter.from_filename(notebook_file_name)
new_fnw = report_file_name
codecs.open(new_fnw, 'w',encoding='utf-8').write(output)